In [69]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

# Wrangle OpenStreetMap Data:

For this particular project ,I have choosed to wrangle the data for india's capital New Delhi.My reason for choosing delhi dataset not only because i know this place, but also the large number of and varied locations of delhi can offer me with challenging data.

### Problems encountered in your map:

I have choosed to clean the following two fields:
1. Street name
2. Post Code

Specifically while talking for mapping two problem are encountered one with each field i choosed to clean.
1. For street name somehow I have to map the abbreviation which are being used with the comprehensive name, and there are some street name specifically with house no. which was again not desirable. so i have constructed mapping in a way that i should take care of both.

In [70]:
mapping = { "St": "Street",
            "St.": "Street",
            "Ave" : "Avenue",
            "Ave." : "Avenue",
            "Rd" : "Road",
            "Rd." : "Road",
            "Blvd" : "Boulevard",
            "Blvd." : "Boulevard",
            "Cir" : "Circle",
            "Cir." : "Circle",
            "Ct" : "Court",
            "Ct." : "Court",
            "Dr" : "Drive",
            "Dr." : "Drive",
            "Pl" : "Place",
            "Pl." : "Place",
            "h/no":"",
            "Vill.":"Village",
            "wz-10":"",
            "B-3/27":"",
            "P.O.":"Post Office"
            }

3. For postal code, I was not sure whether the postcode are valid or not.So I have decided to use a list of valid postcode (source from govt. website). and then mapping the postcode which exists in osm dataset with the valid postcode list.

#### Overview of Data:

Osm Dataset contains mainly following tags.
1. Node
2. Way
3. tag
4. Relation
5. nd
6. memeber
7. osm

Out of all of above type of tags. My analysis is mainly concerned on Node , way and tag tags.

Node and ways tag Data contains following info:

1. lattitue and longitute.
2. User creation records.
3. Name of particular location.
4. Type of place it is.
5. Address if entered.
6. amenity on particular location.
7. Other info like (opening time , closing time) depends upon type of amenity.
8. Refs in way tag data.

I have choosen street names and postcode to clean for this project.following are my observation for street name data and postcode data.


#### Street data observation:

1. Street Names ending with the abbreviation.
2. Street Names are having muliple comma(usually i have observed that people put their complete address in   streetname itself of format like (hno 25,adarsh nagar, 123423).
3. Street Names are ending with postcode number(e.g adarsh nagar 123456)

#### Post Code data observation:

1. Postcode are having character append to them e.g(E123456).
2. Postcode are not authenticated.

### Code Snippet

Following is the code snippet which i have used to clean the street names and postcode and some other check like lat and long should be float are also taken into consideration.

please refer WrangleOpenDataSet.py for complete code.

Following is a sample for way tag data that that should be converted into json format.

{
  "node_refs": [
    "1384029837", 
    "1384029881", 
    "1384029774"
  ], 
  "created": {
    "changeset": "8907357", 
    "user": "abhinavc", 
    "version": "1", 
    "uid": "488226", 
    "timestamp": "2011-08-03T07:15:34Z"
  }, 
  "visible": null, 
  "address": {
    "interpolation": "all"
  }, 
  "type": "way", 
  "id": "124318450"
}
{

Following is a sample for Node tag data that that should be converted into json format.

{
  "building": "yes", 
  "name": "Tata Value Homes", 
  "visible": null, 
  "created": {
    "changeset": "45990828", 
    "user": "Munish Sharma", 
    "version": "1", 
    "uid": "5294790", 
    "timestamp": "2017-02-11T07:14:50Z"
  }, 
  "pos": [
    28.4444554, 
    77.4655984
  ], 
  "phone": "0120 650 0635", 
  "address": {
    "city": "Rajouri Garden Extension, Rajouri Garden, New Delhi", 
    "street": "Greater Noida Expressway", 
    "housenumber": "104, Vardhman Plaza 1, Keshav Marg, Opp Mai Kamli Wali Hospital, Community Centre, Block J, Rajouri Garden, Block J,", 
    "postcode": "201308"
  }, 
  "type": "node", 
  "id": "4679612425"
}

In [71]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import csv
import xlrd
import codecs
import json

#Name of Osm file used for the analysis
OSMFILE = "new-delhi_india_sample.osm"

#Regular expression to find out the invalid street name
street_type_re =  re.compile(r'[\+/&<>;\'"\?%#$@\\.]')

#Regular expression to find out street names ending with the postal code
street_with_postal=re.compile(r'\d{6}$')

#Regular expression to find the valid postal codes.
post_code_re=re.compile(r'\d{6}$')

valid_postcode=set()

#valid postal code file
datafile="ListofPinCodesofDelhi.xls"

#list of created dictionary
CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

#mapping dictionary for problematic data
mapping = { "St": "Street",
            "St.": "Street",
            "Ave" : "Avenue",
            "Ave." : "Avenue",
            "Rd" : "Road",
            "Rd." : "Road",
            "Blvd" : "Boulevard",
            "Blvd." : "Boulevard",
            "Cir" : "Circle",
            "Cir." : "Circle",
            "Ct" : "Court",
            "Ct." : "Court",
            "Dr" : "Drive",
            "Dr." : "Drive",
            "Pl" : "Place",
            "Pl." : "Place",
            "h/no":"",
            "Vill.":"Village",
            "wz-10":"",
            "B-3/27":"",
            "P.O.":"Post Office"
            }




#function to load the valid postcode master data
def load_pin_code_master_data(datafile):   
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)
    for r in range(sheet.nrows-1):
        valid_postcode.add(sheet.cell_value(r+1,2))
    return valid_postcode


                    
#function to update the problematic postcode with the valid ones.
def update_post_code(postcode):
    if post_code_re.search(str(postcode)):
        valid_postcode=load_pin_code_master_data("ListofPinCodesofDelhi.xls")
        if float(postcode[-6:]) in valid_postcode:
            return postcode[-6:]

        
#function to update the problematic street name with the valid ones.
def update_street_name(name, mapping):  
    ll=[]
    updName=""
    if street_type_re.search(name):   
        for k,v in mapping.iteritems():
            if k in name and "," not in name:
                updName= name.replace(k,v)
                if street_with_post_code(updName):
                    updName=street_with_post_code(name)               
            elif k in name and "," in name:
                name1= name.replace(k,v)
                ll=name1.split(",")
                updName=ll[1]
                if street_with_post_code(updName):
                    updName=street_with_post_code(name)    
    elif "," in name:
        ll=name.split(",")
        updName= ll[1]
        if street_with_post_code(updName):
            updName=street_with_post_code(name)  
    elif street_with_post_code(name):
        updName=street_with_post_code(name)
        
    return updName

#function to clean the street name ending with postcode and returning the street name only.
def street_with_post_code(name):
    if street_with_postal.search(name):
        return street_with_postal.sub("",name)

    
#to check whether the given value is a number or not.
def is_number(field_val):
    try:
        float(field_val)
        return True
    except ValueError:
        pass
    
    try:
        import unicodedata as un
        un.numeric(field_val)
        return True
    except(TypeError,ValueError):
        pass

    return False


#function to load the creation dictionary with the valid values.
def load_node_creation_field(element):
    localDic={}
    for attr in element.attrib:
        if attr in CREATED:
            localDic[attr]=element.get(attr)
    return localDic

#function to load the address field after cleaning the street and postcode.
def load_address_field(valfork,valforv):
    ll=[]
    valueforv=""
    if valfork.startswith("addr:"):
        ll=valfork.split(":")
        if len(ll)<3:
            if valfork=="addr:street":
                if update_street_name(valforv, mapping):                       
                    valueforv=update_street_name(valforv, mapping)
                else:
                    valueforv=valforv
            elif valfork=="addr:postcode":
                if update_post_code(valforv):
                    valueforv=update_post_code(valforv)
                    
            else:
                valueforv=valforv
        
    return valueforv


#finally shaping the elements to convert xml file to json.
def shape_element(element):
    node = {}
    createddic={}
    addrdic={}
    longlat=[]
    reflist=[]
    
    #including only node and way tag for analysis.
    if element.tag == "node" or element.tag == "way" :
        #creating longlat entry in desired format.
        if is_number(str(element.get('lat'))) and is_number(str(element.get('lon'))):               
            longlat=[float(element.get('lat')),float(element.get('lon'))]
        
        #shaping creation records
        createddic=load_node_creation_field(element)
        
        #dealing with the address field
        for elem in element.iter("tag"):
            valfork=elem.get("k")
            valforv=elem.get("v")
            if load_address_field(valfork,valforv):
                #print load_address_field(valfork,valforv)
                addrdic[valfork[5:]]=load_address_field(valfork,valforv)                                             
            elif valfork.startswith("name:"):
                ll=valfork.split(":")
                if ll[1]=="en":
                    node[ll[1]]=valforv                    
            else:
                if ":" not in valfork:
                    node[valfork]=valforv
                
        #adding ref records of way tag
        if element.tag=="way":
            for elem1 in element.iter("nd"):
                valforref=elem1.get("ref")
                reflist.append(valforref)
                
        node["type"]=element.tag
        node["id"]=element.get("id")
        node["visible"]=element.get("visible")
        
        #finally adding all the records in desired shape.
        if bool(addrdic):
            node["address"]=addrdic
        if len(longlat)>0:
            node["pos"]=longlat
        if len(reflist)>0:
            node["node_refs"]=reflist
        node["created"]=createddic
        
        return node
    else:
        return None

# converting shaped element to json file.
def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

def main():
    data = process_map(OSMFILE, True)

if __name__ == "__main__":
    main()


After the json file has been created in desired format.then i have to load in json file in mongodb.for loading large file at once in mongo db, i have used the mongo db import functionality.

## Loading Data in MongoDB:

I have used following command to load data in mongoDB

E:\Software\MongoDB\bin>mongoimport --db delhiosm --collection delhiosmcol --drop --file E:/Udacity/P2/new-delhi_india_sample.json

Once the data is loaded in mongoDB I have carried out the various anaysis by using the following code snippet.
Please refer to DataAnalysisWithMongoDB.py for complete code.

In [72]:
import pprint

def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

db = get_db('delhiosm')

#function to give unique record for a particular field.
def unique_record(db,fieldName):
    usercol=db.delhiosmcol.distinct(fieldName)
    return len(usercol)

#function to find total number of records in a file.
def size_of_file(db):
    size=db.delhiosmcol.find().count()
    return size

#function to find number of record cound with for a particular field and value.
def record_count(db,fieldName,valueName):
    rec_count=db.delhiosmcol.find({fieldName:valueName}).count()
    return rec_count


#function to create grouping pipeline for a given field and filter values.
def grouping_pipeline(db,groupByField,filterField,filterValue):
    pipeline=[{"$match":{filterField:filterValue}},
        {"$group":{"_id":groupByField,
                        "count":{"$sum":1}}},
             {"$sort":{"count":-1}}]
    return pipeline

#function specifically for a pipeline to count the number of place lie and long and lat co-ordinates after
#rounding them to two decimal places.
def lonlat_pipline():
    pipeline=[ {"$match":{"pos":{"$exists":1}}},
        {"$project":{"lon":{"$arrayElemAt":["$pos",0]},
               "lat":{"$arrayElemAt":["$pos",1]}}},

              {"$project":{"roundlon" : {
              "$subtract":[
                {"$add":['$lon',0.0049999999999999999999999]},
                {"$mod":[{"$add":['$lon',0.0049999999999999999999999]}, 0.01]}
                          ]
                        },
                          "roundlat" : {
            "$subtract":[
                {"$add":['$lat',0.0049999999999999999999999]},
                {"$mod":[{"$add":['$lat',0.0049999999999999999999999]}, 0.01]}
                  ]
                    } }},
           {"$group":{"_id":{"lat":"$roundlon","lon":"$roundlat"}
                        ,"count":{"$sum":1}}},
            {"$sort":{"count":-1}} ,
            {"$limit":5}
             ]
    return pipeline
    


    

def aggregate(db, pipeline):
    return [doc for doc in db.delhiosmcol.aggregate(pipeline)]




#size of the file
print ("size of file "+str(size_of_file(db)))

#no of unique user
uniqueUserCount=unique_record(db,"created.user")
print ("no. of unique users "+str(uniqueUserCount))

#no of nodes count
nodeCount=record_count(db,"type","node")
print ("no. of nodes count "+str(nodeCount))

#no of ways count
waysCount=record_count(db,"type","way")
print ("no. of ways count "+str(waysCount))

#no of fields having amenity
noOfAmenity=record_count(db,"amenity",{"$exists":1})
print ("no. of record with amenity field "+str(noOfAmenity))

#no of amenity with fast food
noOfAmenity=record_count(db,"amenity","fast_food")
print ("no. of record with fast_food amenity "+str(noOfAmenity))

#count of each amenity
pipeline = grouping_pipeline(db,"$amenity","amenity",{"$exists":1})
result = aggregate(db, pipeline)
pprint.pprint(result)

#no of amenties per religion
pipeline = grouping_pipeline(db,"$religion","amenity","place_of_worship")
result = aggregate(db, pipeline)
print ("no. of amenities per religion")
pprint.pprint(result)

#top 5 records which lie in the nearby position
#I take long and lat rounding till 2 decimal places then print the count of no of places fall on particular co-ordinates.
pipeline = lonlat_pipline()
result = aggregate(db, pipeline)
print ("Top five lattitude and longitute position which maximum nearby places.")
pprint.pprint(result) 



size of file 41001
no. of unique users 377
no. of nodes count 34061
no. of ways count 6940
no. of record with amenity field 50
no. of record with fast_food amenity 1
[{u'_id': u'school', u'count': 12},
 {u'_id': u'place_of_worship', u'count': 6},
 {u'_id': u'parking', u'count': 4},
 {u'_id': u'restaurant', u'count': 4},
 {u'_id': u'fuel', u'count': 4},
 {u'_id': u'bus_station', u'count': 3},
 {u'_id': u'cafe', u'count': 2},
 {u'_id': u'atm', u'count': 2},
 {u'_id': u'bank', u'count': 2},
 {u'_id': u'bar', u'count': 2},
 {u'_id': u'shelter', u'count': 1},
 {u'_id': u'grave_yard', u'count': 1},
 {u'_id': u'post_office', u'count': 1},
 {u'_id': u'fire_station', u'count': 1},
 {u'_id': u'telephone', u'count': 1},
 {u'_id': u'pub', u'count': 1},
 {u'_id': u'fast_food', u'count': 1},
 {u'_id': u'hospital', u'count': 1},
 {u'_id': u'college', u'count': 1}]
no. of amenities per religion
[{u'_id': None, u'count': 3},
 {u'_id': u'jain', u'count': 1},
 {u'_id': u'sikh', u'count': 1},
 {u'_id': u'

### Suggestion for improving the dataset:

After further analysing the data i have found that we can improve dataset by put some accuracy on the is_in field, phone number field and addr:housenumber field

### Is_in field analysis:

1. We can collate a list of valid cities in delhi and then we can validate the value for is_in field with valid city list.
2. There are some values which such as ('National Capital Region, NCR, India), which needs to be cleaned as NCR is an acronym for National Capital Region. and also we sould not have country name in this field. so we can clean that also.

Following is the example dataset that we can clean.

In [73]:
for a in db.delhiosmcol.find({"is_in":{"$exists":1}}):
    pprint.pprint(a)

{u'_id': ObjectId('59338407d220d31c4b7fff29'),
 u'admin_level': u'2',
 u'capital': u'yes',
 u'created': {u'changeset': u'46299189',
              u'timestamp': u'2017-02-22T08:42:55Z',
              u'uid': u'3029661',
              u'user': u'saikabhi',
              u'version': u'53'},
 u'en': u'New Delhi',
 u'id': u'16173236',
 u'is_capital': u'country',
 u'is_in': u'National Capital Region, NCR, India',
 u'name': u'New Delhi',
 u'place': u'city',
 u'population': u'249998',
 u'pos': [28.6138967, 77.2159562],
 u'type': u'node',
 u'visible': None,
 u'wikidata': u'Q987',
 u'wikipedia': u'en:New Delhi'}
{u'_id': ObjectId('5933840cd220d31c4b8084a8'),
 u'created': {},
 u'id': u'71651922',
 u'is_in': u'Noida',
 u'landuse': u'industrial',
 u'layer': u'-5',
 u'name': u'Sector 58',
 u'node_refs': [u'852047558',
                u'852047557',
                u'852047597',
                u'852047705',
                u'852047743',
                u'852047648',
                u'852047677',
    

Distinct value for the is_in field.

In [74]:
db.delhiosmcol.distinct("is_in")

[u'National Capital Region, NCR, India', u'Noida', u'Rohini']

### Benefits of auditing is_in field:

After valiating the accuracy of the solution against a standard dataset of all the cities in delhi.we can clean is_in field with junk data.

### Challenge we might face during implementing solution and how we can solve them:

Following challenges might be faced during solution implementation with possible solution:
1. is_in field can also have the city name containing commas. we have to take care of, by spliting each string  separated by comma and then validating with the standard city dataset. if the a match found in any of the splitted field then only consider that split as a valid city.
2. is_in field might also have more than one splitted field as a valid city.In this particular case we should use postcode for the particular record and try to find a matching records on the basis of postcode in our dataset.Once the matching record found then we can check whether the matched record contains the valid city name, If yes then we can take the valid city of matching record as the city of our record also.
3. We might not valid city after using solution for 2nd challenge also. then we can still search for the second match with postcode . if still no valid value found for the is_in field then finally we can put Delhi as a default city name in problematic record.

### Improving addr:housenumber field.

1. There are some housenumber field containing house number and some additional string data which generally gives some kind of landmark and needs to be cleaned.
2. There are also chances of wrong house number. for this we can check the accuracy by collating a list of valid housenumber in particular area and then validating housenumber against that standard dataset.

### Benefits of auditing housenumber field:

After cleaning the unneccessary string information and checking accuracy against standard housenumber dataset against particular area ,we will left with the valid housenumber.

### Challenge we might face during implementing solution and how we can solve them:

1. We can also have the valid housenumber with alphabets (like A53) ,for taking care of these kind of challenges with should first check whether the housenumber lies in the standard dataset for housenumber , if yes then we should not clean the aphabets from the housenumber, if no the we should first clean the alphabets from house number then again validate the housenumber against the valid data set for housenumbers.
2. We might also have the commas, in the housenumber field. for these cases we should split the string on the basis of comma and then validating each split against the valid housenumber dataset. and keeping the matched one.

In [75]:
for a in db.delhiosmcol.find({"housenumber":{"$regex":"^[[:alnum:]]+$"}}):
    pprint.pprint(a)